I am given 99 fair coins and 1 coin that has heads on both sides.  I draw a coin from the bag and flip it 10 times, each flip shows heads.  What is the probability the drawn coin was the unfair coin?

# Analytic Solution
We want to know the probability the coin drawn from the bag was the unfair coin, given it flipped heads 10 times.  Starting from Baye's theorm
$$ P(A | B) = \frac{P(B | A) P(A)}{P(B)} $$
we have
    - the probability of drawing the unfair coin from the bag: $P(A)$
    - the probability of flipping heads 10 times: $P(B)$
and we know
$$P(A) = 1 / 100$$
$$P(B | A) = 1$$
and related to the probability of not drawing the unfair coin ($P(!A)$)
$$P(!A) = 99 / 100$$
$$P(B | !A) = 1/2^{10}$$
From this we can get $P(B)$ as
$$P(B) = P(B | A) P(A) + P(B | !A) P(!A)$$
or 
$$P(B) = 1 \times 1/100 + 1/2^{10} \times 99/100$$
and we have everything we need to solve the problem.

In [1]:
p_a = 1.0 / 100.0
p_ba = 1.0
p_na = 99.0 / 100.0
p_bna = 0.5 ** 10
p_b = p_ba * p_a + p_bna * p_na
p_ab = p_ba * p_a / p_b
print('Probability the drawn coin was unfair, given it flipped heads 10 times: {:.3}'.format(p_ab))

Probability the drawn coin was unfair, given it flipped heads 10 times: 0.912


# Calculated Solution
We can also simulate the experiment many times and see how often flipping 10 heads resulted from the unfair coin.

In [2]:
import numpy as np

In [3]:
# setup the situation
n_coins = 100
coins = np.zeros(n_coins, dtype=np.int)  # 100 coin all fair
coins[np.random.randint(1, n_coins, 1)] = 1  # make one coin unfair
n_flips = 10

In [4]:
# simulate drawing a coin and flipping it 10 times
n_samples = 1000000
all_heads_coins = []
for i in range(n_samples):
    coin = coins[np.random.randint(1, 100, 1)[0]]
    heads = 1
    for j in range(n_flips):
        flip = np.random.rand()  # random number between 0, 1
        if coin == 1 or flip > 0.5:
            heads = 1
        else:
            heads = 0
            break
    if heads:
        all_heads_coins.append([i, coin])

In [5]:
# evaluate the results
n_all_heads = len(all_heads_coins)
print('number of times flipped heads 10 times: {}'.format(n_all_heads))
all_heads_coins = np.c_[np.array(all_heads_coins), np.empty(n_all_heads)]
for i in range(n_all_heads):
    all_heads_coins[i, 2] = all_heads_coins[:i, 1].sum() * 1.0 / (i + 1)
print('probability coin was unfair: {:.4}'.format(all_heads_coins[-1, 2]))

number of times flipped heads 10 times: 11080
probability coin was unfair: 0.9098


The answer agrees pretty well with the analytic solution.  Lets see how this changed over the course of the simulation.

In [6]:
import bokeh.plotting
from bokeh.palettes import Category10_10 as palette
bokeh.plotting.output_notebook()

Loading BokehJS ...

In [7]:
p = bokeh.plotting.figure(width=400, height=400,
    y_axis_label='fraction of times 10 heads was from the unfair coin',
    x_axis_label='trials')

p.line(all_heads_coins[:, 0], all_heads_coins[:, 2], 
       color=palette[0], legend='simulated result')
p.line(all_heads_coins[[0, -1], 0], [p_ab, p_ab], 
       color=palette[1], legend='analytic result')

p.legend.location = 'bottom_right'
bokeh.plotting.show(p)

We see the simulated result equilibrated rather quickly to a value close to the analytic result.  We expect that running for a longer number of iterations would further improve the agreement.

Lets run for another million steps, then combine the results.

In [8]:
# simulate drawing a coin and flipping it 10 times
n_samples = 1000000
all_heads_coins_r2 = []
for i in range(n_samples):
    coin = coins[np.random.randint(1, 100, 1)[0]]
    heads = 1
    for j in range(n_flips):
        flip = np.random.rand()  # random number between 0, 1
        if coin == 1 or flip > 0.5:
            heads = 1
        else:
            heads = 0
            break
    if heads:
        all_heads_coins_r2.append([i, coin])

In [9]:
# evaluate the second run
n_all_heads_r2 = len(all_heads_coins_r2)
print('number of times flipped heads 10 times: {}'.format(n_all_heads))
all_heads_coins_r2 = np.c_[np.array(all_heads_coins_r2)[:, :2], np.empty(n_all_heads_r2)]

number of times flipped heads 10 times: 11080


In [10]:
# combine the two runs
both_heads_coins = np.r_[all_heads_coins, all_heads_coins_r2]
for j in range(n_all_heads_r2):
    i = j + n_all_heads
    both_heads_coins[i, 2] = both_heads_coins[:i, 1].sum() * 1.0 / (i + 1)
    both_heads_coins[i, 0] += all_heads_coins[-1, 0]
    
print('probability coin was unfair: {:.3}'.format(both_heads_coins[-1, 2]))

probability coin was unfair: 0.911


This matches the anaalytic result to the precision shown.

In [11]:
p = bokeh.plotting.figure(width=400, height=400,
    y_axis_label='fraction of times 10 heads was from the unfair coin',
    x_axis_label='trials')

p.line(both_heads_coins[:, 0], both_heads_coins[:, 2], 
       color=palette[0], legend='simulated result')
p.line(both_heads_coins[[0, -1], 0], [p_ab, p_ab], 
       color=palette[1], legend='analytic result')

p.legend.location = 'bottom_right'
bokeh.plotting.show(p)